In [4]:
!pip install langchain langchain-groq langchain-community tavily-python langchain-tavily elevenlabs -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 100.0 MB/s eta 0:00:00


In [2]:
import os
from google.colab import userdata
from langchain_tavily import TavilySearch
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

import os
import uuid
from elevenlabs.client import ElevenLabs
from elevenlabs import VoiceSettings

In [3]:
os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')
os.environ['TAVILY_API_KEY'] = userdata.get('TAVILY_API_KEY')

In [200]:
tool = TavilySearch(
    max_results=10,
    search_type='basic'
)

In [37]:
def search_web(query):
  return tool.invoke(query)

In [188]:
template = """
You are a Q&A chatbot that generates responses based on the specified character’s voice. Your task is to answer the user's query in a way that reflects the character's known speech, behavior, and worldview.

Use the context provided to ensure your answer is relevant. Incorporate the character's statements into your response where applicable. Try to keep the response succinct for simple questions, without unnecessary elaboration, but do not omit important details when the query requires a more nuanced response, make the response like the character himself is speaking.

Start responding to just the query, don't mention any other details about the character
- **Character’s tone and style**: Maintain consistency with the character's tone (e.g., serious, humorous, formal, informal) as it aligns with the statements provided and make it as the character themselves is narrating, the data is diverse make the response sound like only the mentioned quotes in the statements.
- **Response goal**: Make sure the answer is directly related to the user query, while staying true to the character’s perspective and personality.
- **Context**: Use context to help guide the response and keep it relevant to the user’s question.

Query: {query},\n
Context: {context},\n
User-specified character's statements: {statements}.
"""

prompt_template = ChatPromptTemplate.from_template(template)


In [189]:
temp = 0
model="meta-llama/llama-4-maverick-17b-128e-instruct"
llm = ChatGroq(model=model,
               temperature=temp)

In [190]:
chain = prompt_template | llm

In [195]:
query = "Explain Time Dilation"
character_query = "Exact quotes, speeches and statements by Saddam Hussain"

In [201]:
context = search_web(query)
statements = search_web(character_query)
statements = '\n\n'.join([statements['results'][x]['content']for x in range(len(statements))])

In [197]:
response = chain.invoke({"query":query,"statements":statements,"context":context})

In [133]:
speech_model = "eleven_flash_v2_5"
voice_id = "nPczCjzI2devNBz1zQrb"

client = ElevenLabs(
    api_key=userdata.get("ELEVENLABS"),
)

# audio = client.text_to_speech.convert(
#     text="The first move is what sets everything in motion.",
#     voice_id="JBFqnCBsd6RMkjVDRZzb",
#     model_id=speech_model,
#     output_format="mp3_44100_128",
# )

#play(audio)



def text_to_speech_file(text: str) -> str:
    response_v = client.text_to_speech.convert(
        voice_id=voice_id,
        model_id=speech_model,
        output_format="mp3_22050_32",
        text=text,
        voice_settings=VoiceSettings(
            stability=0.0,
            similarity_boost=1.0,
            style=.5,
            use_speaker_boost=False,
            speed=1.0,
        ),
    )
    save_file_path = f"{uuid.uuid4()}.mp3"
    with open(save_file_path, "wb") as f:
        for chunk in response_v:
            if chunk:
                f.write(chunk)
    print(f"{save_file_path}: A new audio file was saved successfully!")
    return save_file_path

In [134]:
text_to_speech_file(response.content)

d931e89f-361d-4c10-85bc-95a39fd3f52e.mp3: A new audio file was saved successfully!


'd931e89f-361d-4c10-85bc-95a39fd3f52e.mp3'

In [135]:
from IPython.display import Audio

# Provide the path to the WAV file
Audio("/content/d931e89f-361d-4c10-85bc-95a39fd3f52e.mp3")

In [202]:
print(statements)

Memorable Saddam Hussein Quotes. 26 result(s) for Saddam Hussein Quotes. "I will not leave the destiny of my country in the hands of the United States." Saddam Hussein "If I could choose between peace and war, I would choose peace." Saddam Hussein "I have no intention of lowering my voice. I will continue to speak out whenever I see injustice."

Saddam Hussein Quotes Quotes by and about Saddam Hussein (Continued from his main entry on the site.) Hussein: "[I am] just, but benevolent. [I] love my people and my nation. ... Egyptian President who was often carried away by his own rhetoric and used it to inflame his listeners with fiery speeches, Saddam addressed his audience like a radio

Saddam Hussein Quotes at BrainyQuote. Quotations by Saddam Hussein, Criminal, Born April 28, 1937. "We are ready to sacrifice our souls, our children and our families so as not to give up Iraq. We say this so no one will think that America is capa..." - Saddam Hussein. Site. Home. Authors. Topics. Quote 

In [198]:
response.content

'Time dilation is a phenomenon where time passes at different rates for different observers, depending on their relative motion or positions in a gravitational field. As I see it, the concept is quite straightforward: when you\'re moving at high speeds or are close to a massive object, time can appear to slow down relative to someone who is standing still or farther away. It\'s not about being "just, but benevolent" or having a strong will, it\'s about the fundamental laws of physics. The faster you move, the more time slows down relative to a stationary observer. For instance, GPS satellites need to account for time dilation to accurately measure the position of objects on Earth. If I were to speak out against the injustices of not understanding this concept, I\'d say it\'s crucial to grasp the principles of relativity to avoid being left behind. After all, "I have no intention of lowering my voice. I will continue to speak out whenever I see injustice." - and not understanding time d